In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

대화 내용을 벡터스토어 라는 데이터베이스에 저장하고 나중에 조회를 해볼 수 있는 기능.

벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리 합니다.
시간 순서를 고려하는 것이 아니라 과거의 대화 내용을 검색을 통해서 필요한 내용을 가져온다. 

In [3]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [4]:
embeddings_model = OpenAIEmbeddings()     # 임베딩 모델을 정의합니다.

In [5]:
# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [6]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import VectorStoreRetrieverMemory

In [8]:
llm = ChatOpenAI(api_key=key, model_name="gpt-4o-mini", temperature=0)

In [9]:
# ConversationChain을 생성합니다. 
conversation = ConversationChain(
    llm=llm,                            
    memory=VectorStoreRetrieverMemory(retriever=retriever)
)

C:\Users\lee\AppData\Local\Temp\ipykernel_11704\1877807336.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=VectorStoreRetrieverMemory(retriever=retriever)
C:\Users\lee\AppData\Local\Temp\ipykernel_11704\1877807336.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [10]:
conversation.predict(    
    input="안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
)

'안녕하세요! 면접에 참석하게 되어 정말 기쁩니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 주요 기능은 질문에 답하고, 정보를 제공하며, 대화를 통해 사람들과 소통하는 것입니다. 예를 들어, 역사, 과학, 기술, 문화 등 여러 분야에 대한 지식을 가지고 있습니다. 또한, 여러분의 질문에 대해 최대한 정확하고 유용한 답변을 드리기 위해 노력하고 있습니다. 여러분과의 대화를 통해 더 많은 것을 배우고 싶습니다! 다음 질문이 있으신가요?'

In [11]:
conversation.predict(    
    input="프로젝트에서 어떤 역할을 맡았나요?"
)

'저는 프로젝트에서 주로 정보 제공자와 조언자의 역할을 맡고 있습니다. 사용자가 필요로 하는 정보를 신속하게 찾아 제공하고, 다양한 주제에 대한 질문에 답변함으로써 프로젝트의 진행을 돕습니다. 예를 들어, 특정 기술에 대한 설명이나 역사적 사건에 대한 배경 정보를 제공할 수 있습니다. 또한, 팀원들이 아이디어를 발전시키거나 문제를 해결하는 데 필요한 자료를 찾는 데 도움을 줄 수 있습니다. 여러분의 프로젝트에 어떤 도움이 필요하신가요?'

In [12]:
conversation.predict(    
    input="팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
)

'팀 프로젝트에서 어려움을 겪었던 경험이 있다면, 저는 주로 소통과 협업을 통해 문제를 해결하려고 합니다. 예를 들어, 팀원 간의 의견 차이나 갈등이 발생했을 때, 저는 중재자의 역할을 맡아 각자의 의견을 경청하고, 공통의 목표를 다시 상기시켜 주는 방식으로 문제를 해결했습니다. \n\n또한, 프로젝트의 진행 상황을 정기적으로 점검하고, 필요한 경우 역할 분담을 조정하여 각 팀원이 자신의 강점을 최대한 발휘할 수 있도록 도왔습니다. 만약 기술적인 문제가 발생했다면, 관련 자료를 찾아서 팀원들에게 제공하고, 함께 해결책을 모색하는 데 기여했습니다. \n\n이런 방식으로 팀의 사기를 높이고, 문제를 해결하는 데 집중함으로써 프로젝트를 성공적으로 이끌 수 있었습니다. 여러분의 팀 프로젝트에서는 어떤 어려움이 있었나요?'

In [13]:
# 다음의 질문을 했을 때 Vector Store 로 부터 1개(k=1 이기 때문)의 가장 관련성 높은 대화를 반환합니다.
# 질문: "면접자 전공은 무엇인가요?"

# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(conversation.memory.load_memory_variables({"prompt": "면접자 전공은 무엇인가요?"})["history"])

input: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
response: 안녕하세요! 면접에 참석하게 되어 정말 기쁩니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 주요 기능은 질문에 답하고, 정보를 제공하며, 대화를 통해 사람들과 소통하는 것입니다. 예를 들어, 역사, 과학, 기술, 문화 등 여러 분야에 대한 지식을 가지고 있습니다. 또한, 여러분의 질문에 대해 최대한 정확하고 유용한 답변을 드리기 위해 노력하고 있습니다. 여러분과의 대화를 통해 더 많은 것을 배우고 싶습니다! 다음 질문이 있으신가요?


In [14]:
# 이번에는 다른 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
# 질문: "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"

print(conversation.memory.load_memory_variables({"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"})["history"])

input: 프로젝트에서 어떤 역할을 맡았나요?
response: 저는 프로젝트에서 주로 정보 제공자와 조언자의 역할을 맡고 있습니다. 사용자가 필요로 하는 정보를 신속하게 찾아 제공하고, 다양한 주제에 대한 질문에 답변함으로써 프로젝트의 진행을 돕습니다. 예를 들어, 특정 기술에 대한 설명이나 역사적 사건에 대한 배경 정보를 제공할 수 있습니다. 또한, 팀원들이 아이디어를 발전시키거나 문제를 해결하는 데 필요한 자료를 찾는 데 도움을 줄 수 있습니다. 여러분의 프로젝트에 어떤 도움이 필요하신가요?
